In [1]:
import spimage
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [7]:
runnr = 98
base_path = '/Users/benedikt/data/LCLS/amol3416/pbcv/'
filename_ol1 = base_path + 'amol3416_r%04d_ol1.h5' %runnr
filename_ol3 = base_path + 'amol3416_r%04d_ol3.h5' %runnr
filename_sizing  = base_path + 'sizing/amol3416_r%04d_sizing.h5' %runnr
filename_phasing = base_path + 'phasing_threshold/amol3416_r%04d_phasing.h5' %runnr
outfolder = base_path + 'emc/input/'

# Save scaled thumbnails (for tagging)

In [3]:
with h5py.File(filename_phasing, 'a') as f:
    thumbnail = np.abs(f['entry_1/result_1/thumbnail'][:])
    m = thumbnail.sum(axis=(1,2)).reshape((thumbnail.shape[0], 1,1))
    thumbnail_normalized = thumbnail / m * m.mean()
    del f['entry_1/result_1/thumbnail2']
    f['entry_1/result_1/thumbnail2'] = thumbnail_normalized
    f['entry_1/result_1/thumbnail2'].attrs["axes"] = ['experiment_identifier:y:x']

# Extract good data and save for emc

In [4]:
with h5py.File(filename_phasing, 'r') as f:
    good = f['entry_1/result_1/tags'][0] == False
    intensities = f['entry_1/result_1/intensities'][good,:,:]
    real_space = np.abs(f['entry_1/result_1/real_space'][good,:,:])
    mask = f['entry_1/result_1/mask'][good,:,:]

In [16]:
scaling = np.sum(real_space**2, axis=(1,2)).reshape((real_space.shape[0],1,1))
intensities_scaled = intensities / scaling * scaling.mean()

In [26]:
distance = 0.37 #[m]
wavelength = 1.59e-9 #[m]
center = (128.,128.)
pixelsize = 4*75e-6 # [m]
nimages = intensities_scaled.shape[0]

In [51]:
for i in range(nimages):
    with h5py.File(outfolder + '%04d.h5' %i, 'w') as f:
        f['detector_distance'] = [distance]
        f['image_center'] = center
        f['lambda'] = [wavelength]
        f['mask'] = mask[i]
        f['num_dimensions'] = [2]
        f['phased'] = [0]
        f['pixel_size'] = [pixelsize]
        f['real'] = intensities_scaled[i]
        f['scaled'] = [0.]
        f['shifted'] = [0.]
        f['version'] = [2]

In [52]:
with h5py.File(outfolder + 'mask.h5', 'w') as f:
    f['detector_distance'] = [distance]
    f['image_center'] = center
    f['lambda'] = [wavelength]
    f['mask'] = np.alltrue(mask, axis=0)
    f['num_dimensions'] = [2]
    f['phased'] = [0]
    f['pixel_size'] = [pixelsize]
    f['real'] = np.alltrue(mask, axis=0)
    f['scaled'] = [0.]
    f['shifted'] = [0.]
    f['version'] = [2]